In [1]:
import numpy as np
import pandas as pd

file_path = '/content/sample_data/Edges.edges'

# خواندن فایل به عنوان DataFrame
dft = pd.read_csv(file_path, sep='\t')



# نمایش داده‌ها به صورت یک جدول DataFrame
display(dft)

,1,42325
0,2,94060
1,2,66010
2,3,74093
3,3,29846
4,4,96512
...,...,...
437245,99999,62893
437246,100000,43369
437247,100000,88461
437248,100000,33723


In [2]:
file_path = '/content/sample_data/Node_labels.node_labels'

# خواندن فایل به عنوان DataFrame
df = pd.read_csv(file_path, sep='\t')

# نمایش داده‌ها به صورت یک جدول DataFrame
display(df)
print(df.columns)

,1,8
0,2,9
1,3,2
2,4,9
3,5,6
4,6,1
...,...,...
92476,92478,8
92477,92479,5
92478,92480,9
92479,92481,6


Index(['1', '8'], dtype='object')


In [3]:
from sklearn.model_selection import train_test_split



# تعداد نمونه‌ها از هر برچسب را محاسبه می‌کنیم
label_counts = df['8'].value_counts()
print(label_counts)
# تعداد نمونه‌هایی که باید از هر برچسب انتخاب شود
samples_per_label = 1000 // len(label_counts)

# انتخاب نمونه‌ها
sampled_df = pd.DataFrame()
for label in label_counts.index:
    label_df = df[df['8'] == label]
    sampled_label_df, _ = train_test_split(label_df, train_size=samples_per_label, random_state=42)
    sampled_df = pd.concat([sampled_df, sampled_label_df])

# اگر تعداد نمونه‌های انتخاب شده کمتر از 1000 است، بقیه را به صورت تصادفی انتخاب می‌کنیم
if len(sampled_df) < 1000:
    remaining_samples = 1000 - len(sampled_df)
    remaining_df = df.drop(sampled_df.index)
    additional_samples_df, _ = train_test_split(remaining_df, train_size=remaining_samples, random_state=42)
    sampled_df = pd.concat([sampled_df, additional_samples_df])

# حالا sampled_df شامل 1000 راس انتخاب شده است که همه برچسب‌ها را پوشش می‌دهد
display(sampled_df)

8
2    10276
6    10276
1    10276
3    10276
5    10276
4    10276
7    10276
8    10275
9    10274
Name: count, dtype: int64


,1,8
73297,73299,2
24771,24773,2
81299,81301,2
22229,22231,2
64790,64792,2
...,...,...
47045,47047,9
48546,48548,9
7710,7712,9
65600,65602,9


In [4]:
import pandas as pd
import networkx as nx
from collections import defaultdict
from sklearn.cluster import SpectralClustering

# فرض می‌کنیم df_edges نام دیتافریم حاوی یال‌ها و df_nodes نام دیتافریم حاوی راس‌ها و برچسب‌هایشان است
sampled_df_numeric = df.select_dtypes(include=[np.number])
# ساخت گراف خالی
G = nx.Graph()

# اضافه کردن راس‌ها به گراف
for index, row in sampled_df_numeric.head(2000).iterrows():
    G.add_node(row['1'], label=row['8'])

# محاسبه درجه هر راس
degrees = defaultdict(int)
for node in G.nodes():
    degrees[node] = G.degree(node)

# اضافه کردن یال‌ها و وزن‌های مربوطه به گراف
for index, row in dft.head(2000).iterrows():
    node_start = row['1']
    node_end = row['42325']
    # محاسبه میانگین درجه دو راس
    weight = (degrees[node_start] + degrees[node_end]) / 2
    G.add_edge(node_start, node_end, weight=weight)

# تعداد خوشه‌ها را تعیین کنید
num_clusters = 3  # به عنوان مثال، ما 3 خوشه را انتخاب کرده‌ایم

# ماتریس مجاورت گراف G را بدست آورید
adj_matrix = nx.to_numpy_array(G)
# ایجاد یک نمونه از SpectralClustering با تعداد خوشه‌های مورد نظر
sc = SpectralClustering(num_clusters, affinity='precomputed', n_init=100)

# انجام خوشه‌بندی طیفی
sc.fit(adj_matrix)

# بدست آوردن برچسب‌های خوشه برای هر راس
cluster_labels = sc.labels_
for i, node in enumerate(G.nodes()):
    G.nodes[node]['cluster'] = cluster_labels[i]
cluster_data = []

# پر کردن لیست با شناسه‌های راس و برچسب‌های خوشه
for node, data in G.nodes(data=True):
    cluster_data.append([node, data['cluster']])

# تبدیل لیست به DataFrame
cluster_df = pd.DataFrame(cluster_data, columns=['node_id', 'cluster_label'])

# نمایش جدول خوشه‌ها
display(cluster_df)

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


,node_id,cluster_label
0,2,0
1,3,0
2,4,0
3,5,0
4,6,0
...,...,...
3851,14526,0
3852,35582,0
3853,56853,0
3854,41800,0
